In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import os
import numpy as np
import matplotlib.pyplot as plt

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data Paths
data_dir = "data/dataset"

Using device: cuda


In [2]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(150, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])

# Load Dataset using ImageFolder
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)

# Split Dataset into Train & Test (80%-20%)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Data Loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Class Names
class_names = dataset.classes
print(f"Classes: {class_names}")  # ['with_mask', 'without_mask']


Classes: ['with_mask', 'without_mask']


In [68]:
import torch
import torch.nn as nn
import torch.optim as optim

class MaskCNN(nn.Module):
    def __init__(self):
        super(MaskCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 100, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(100, 100, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(100 * 37 * 37, 50)  # Adjusted for image size
        self.fc2 = nn.Linear(50, 2)  # 2 classes: With Mask & Without Mask

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.shape[0], -1)  # Flatten
        # x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class EarlyStopper:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_validation_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_validation_loss - self.min_delta:
            self.best_validation_loss = val_loss
            self.counter = 0
        elif val_loss > self.best_validation_loss + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Assuming 'device', 'train_loader', and 'test_loader' are already defined

# Initialize Model
model = MaskCNN().to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()  # Since we have 2 classes (softmax output)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early Stopping Initialization
patience = 5  # Number of epochs to wait for improvement
min_delta = 0.001 # Minimum change in validation loss to consider as improvement
early_stopper = EarlyStopper(patience=patience, min_delta=min_delta)

# Training the Model
num_epochs = 15  # Increased max epochs to allow for early stopping
train_losses, val_losses = [], []
best_val_loss = float('inf')
best_model_state = None

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation Step
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Accuracy Calculation
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    val_loss /= len(test_loader)
    val_acc = correct / total

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}")

    # Early Stopping Check
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        torch.save(best_model_state, "best_mask_model.pth")

# Load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("Loaded best model weights.")
else:
    print("No improvement found during training, using the last trained model.")

# Optionally save the final model (which might not be the best)
torch.save(model.state_dict(), "final_mask_model.pth")

Epoch [1/15], Train Loss: 0.5640, Val Loss: 0.2472, Val Acc: 0.91
Epoch [2/15], Train Loss: 0.2024, Val Loss: 0.1743, Val Acc: 0.93
Epoch [3/15], Train Loss: 0.1554, Val Loss: 0.1555, Val Acc: 0.94
Epoch [4/15], Train Loss: 0.1354, Val Loss: 0.1523, Val Acc: 0.95
Epoch [5/15], Train Loss: 0.1331, Val Loss: 0.1596, Val Acc: 0.95
Epoch [6/15], Train Loss: 0.1113, Val Loss: 0.1658, Val Acc: 0.94
Epoch [7/15], Train Loss: 0.1003, Val Loss: 0.1521, Val Acc: 0.96
Epoch [8/15], Train Loss: 0.1099, Val Loss: 0.1806, Val Acc: 0.94
Epoch [9/15], Train Loss: 0.1019, Val Loss: 0.1097, Val Acc: 0.96
Epoch [10/15], Train Loss: 0.1036, Val Loss: 0.1293, Val Acc: 0.96
Epoch [11/15], Train Loss: 0.0834, Val Loss: 0.1138, Val Acc: 0.96
Epoch [12/15], Train Loss: 0.0915, Val Loss: 0.1372, Val Acc: 0.94
Epoch [13/15], Train Loss: 0.0812, Val Loss: 0.1226, Val Acc: 0.96
Epoch [14/15], Train Loss: 0.0640, Val Loss: 0.1101, Val Acc: 0.96
Epoch [15/15], Train Loss: 0.0565, Val Loss: 0.1587, Val Acc: 0.96
Earl

In [ ]:
# Final Evaluation on Training Set
model.eval()
correct_train = 0
total_train = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc * 100:.2f}%")

Test Accuracy: 95.67%
